In [1]:
from flask import Flask, Response, stream_with_context
import cv2
import requests
import numpy as np
from ultralytics import YOLO
import pandas as pd
from glob import glob
import matplotlib.pylab as plt
import torch

app = Flask(__name__)

In [2]:
model_type = "MiDaS_small"
midas = torch.hub.load('intel-isl/MiDaS', model_type)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
midas.to(device)
midas.eval()

# Input transformation pipeline
transforms = torch.hub.load('intel-isl/MiDaS', 'transforms')
if model_type == "DPT_Large" or model_type == "DPT_Hybrid":
    transform = transforms.dpt_transform
else:
    transform = transforms.small_transform

Using cache found in /Users/bigfatrat/.cache/torch/hub/intel-isl_MiDaS_master


Loading weights:  None


Using cache found in /Users/bigfatrat/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master
Using cache found in /Users/bigfatrat/.cache/torch/hub/intel-isl_MiDaS_master


In [3]:
def find_largest_max_depth_area(depth_map):
    THRESHOLD = 100
    max_depth = np.max(depth_map)
    print("MAX DEPTH: ", max_depth)
    
    # binary mask (0,1)
    mask = depth_map >= max_depth - THRESHOLD
    mask = mask.astype(np.uint8)
    
    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # key=cv2.contourArea -> identifies the largest connected region 
    largest_contour = max(contours, key=cv2.contourArea) 
    
    # Get bounding box and centroid
    x, y, w, h = cv2.boundingRect(largest_contour)
    centroid_x = x + w // 2
    centroid_y = y + h // 2
    
    return max_depth, (centroid_x, centroid_y), contours



def visualize_largest_max_depth_area(depth_map, rgb_image):
    max_depth, centroid, contours = find_largest_max_depth_area(depth_map)
    
    return rgb_image, max_depth

In [4]:
model = YOLO('../Crater-Detector/best.pt')
class_names = model.names

In [5]:
# URL to receive the video feed from Raspberry Pi
RPI_VIDEO_URL = "http://192.168.107.54:5000/video_feed"  # Replace with actual Raspberry Pi IP

In [6]:
def process_frame(frame):
    # Convert frame to RGB
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Apply depth estimation
    img_batch = transform(img).to(device)
    with torch.no_grad():
        depth_prediction = midas(img_batch)
        depth_prediction = torch.nn.functional.interpolate(
            depth_prediction.unsqueeze(1), size=img.shape[:2], mode='bicubic', align_corners=False
        ).squeeze()
        depth_output = depth_prediction.cpu().numpy()

    # Normalize depth values to 0-255 for visualization
    output = (depth_output - depth_output.min()) / (depth_output.max() - depth_output.min()) * 255
    output = output.astype(np.uint8)
   
    # Visualize the largest max depth area
    image_with_depth, max_depth = visualize_largest_max_depth_area(depth_output, img)

    # Perform YOLO object detection
    results = model(img.copy())

    for r in results:
        boxes = r.boxes
        masks = r.masks

        if masks is not None:
            masks = masks.data.cpu()
            
            for seg, box in zip(masks.data.cpu().numpy(), boxes):
                seg = cv2.resize(seg, (img.shape[1], img.shape[0])) 
                contours, _ = cv2.findContours((seg).astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                sorted_contours = sorted(contours, key=cv2.contourArea, reverse=True)
                # print(sorted_contours)
                for i, contour in enumerate(sorted_contours):
                    M = cv2.moments(contour)
                    if M["m00"] == 0:
                        continue
                    cX = int(M["m10"] / M["m00"])
                    cY = int(M["m01"] / M["m00"])
                    
                    d = int(box.cls)
                    c = class_names[d]
                    x, y, x1, y1 = cv2.boundingRect(contour)

                    depth_at_center = depth_output[cY, cX]  # Get depth at object center

                    ratio_pixel_mm = 155 / 14 # 14 mm = 155 pixels
                    mm = x1 / ratio_pixel_mm
                    cm = mm / 10
                    
                    rect = cv2.minAreaRect(contour)
                    (mx,my),(mw,mh),angle = rect
                    
    
    # Send the max depth value back to the Raspberry Pi server
    requests.post("http://192.168.107.54:5000/processing_done", json={"max_depth": max_depth})
    
    # return img  # Just for completeness, though we don't return this to the server

In [7]:
def generate_processed_frames():
    stream = requests.get(RPI_VIDEO_URL, stream=True)
    byte_buffer = b''
    for chunk in stream.iter_content(chunk_size=1024):
        byte_buffer += chunk

        a = byte_buffer.find(b'\xff\xd8')  # Start of JPEG
        b = byte_buffer.find(b'\xff\xd9')  # End of JPEG

        if a != -1 and b != -1:
            jpg = byte_buffer[a:b+2]
            byte_buffer = byte_buffer[b+2:]

            # Decode frame and process
            frame = cv2.imdecode(np.frombuffer(jpg, dtype=np.uint8), cv2.IMREAD_COLOR)
            processed_frame = process_frame(frame)

            # Send confirmation message back to Raspberry Pi after processing
            requests.post("http://192.168.107.54:5000/processing_done", json={"message": "Processing done for this frame"})

            # Encode the processed frame back to JPEG
            # ret, buffer = cv2.imencode('.jpg', processed_frame)
            # if not ret:
            #     continue

            # processed_frame = buffer.tobytes()

            # # Yield the processed frame as bytes
            # yield (b'--frame\r\n'
            #        b'Content-Type: image/jpeg\r\n\r\n' + processed_frame + b'\r\n')

In [8]:
@app.route('/')
def index():
    # Automatically redirect to /processed_feed
    return Response("Redirecting to /processed_feed", status=302, headers={'Location': '/processed_feed'})

@app.route('/processed_feed')
def processed_feed():
    return Response(stream_with_context(generate_processed_frames()), mimetype='multipart/x-mixed-replace; boundary=frame')

In [9]:
app.run(host='0.0.0.0', port=5001)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://192.168.107.172:5001
Press CTRL+C to quit


MAX DEPTH:  695.22797

0: 480x640 (no detections), 179.7ms
Speed: 1.6ms preprocess, 179.7ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)


[2024-10-25 00:50:31,896] ERROR in app: Exception on /processed_feed [GET]
Traceback (most recent call last):
  File "/Users/bigfatrat/miniconda3/lib/python3.12/site-packages/flask/app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/bigfatrat/miniconda3/lib/python3.12/site-packages/flask/app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/bigfatrat/miniconda3/lib/python3.12/site-packages/flask/app.py", line 880, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/bigfatrat/miniconda3/lib/python3.12/site-packages/flask/app.py", line 865, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/gv/z67rxx